### 필요한 라이브러리

In [ ]:
# !pip install -q accelerate scipy tensorboardX peft bitsandbytes transformers trl tensorboardX

In [ ]:
# !pip install hf_xet

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, # 인과적 언어 추론(예: GPT)을 위한 모델을 자동으로 불러오는 클래스
    AutoTokenizer,        # 입력 문장을 토큰 단위로 자동으로 잘라주는 역할
    BitsAndBytesConfig,   # 모델 구성 (벡터의 INT8 최대 절대값 양자화 기법을 사용할 수 있도록 도와주는 Meta AI의 라이브러리)
    HfArgumentParser,     # 파라미터 파싱
    TrainingArguments,    # 훈련 설정
    pipeline,             # 파이프라인 설정 
    logging,              # 로깅을 위한 클래스
)

# 모델 튜닝을 위한 라이브러리
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

c:\Users\hyunwook\anaconda3\envs\snu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Llama 2 모델과 데이터 로드

In [2]:
# Hugging Face 허브에서 훈련하고자 하는 모델을 가져와서 이름 지정
model_name = "NousResearch/Llama-2-7b-chat-hf"

# instruction 데이터 세트 설정
dataset_name = "mlabonne/guanaco-llama2-1k"

# fine-tuning을 거친 후의 모델에 부여될 새로운 이름을 지정
new_model = "tuned-llama-2-7b-miniguanaco"

### LoRA(Low-Rank Adaptation) 파라미터 설정

In [3]:
# LoRA에서 사용하는 low-rank matrices 어텐션 차원을 정의. 여기서는 64로 설정
# 값이 크면 클수록 더 많은 수정이 이루어지며, 모델이 더 복잡해질 수 있음
lora_r = 16 # sionic ai에선 64
# 값이 작을수록 학습 시간, 리소스 사용량, 학습할 수 있는 표현의 다양성(복잡성) 감소
# 클수록 증가
# 쉽게 말해, 클수록 성능이 좋아지고 메모리 사용량도 증가
# 값이 크면 과적합, 작으면 과소적합
# 기존 모델의 파라미터에 영향을 주는 레이어가 저차원 행렬인듯한데
# 저차원 행렬의 차원을 결정한다.

# LoRA 적용 시 가중치에 곱해지는 스케일링 요소. 여기서는 16으로 설정
# LoRA가 적용될 때 원래 모델의 가중치에 얼마나 영향을 미칠지 결정. 높은 값은 가중치 조정의 강도를 증가시킴
lora_alpha = 16
# 값이 낮으면 기존 데이터를 위주로 높으면 새로운 데이터 위주로 훈련 데이터를 수용

# Dropout probability for LoRA layers
# LoRA 층에 적용되는 드롭아웃 확률. 여기서는 0.1 (10%)로 설정
lora_dropout = 0.05 # 일부 네트워크 연결을 무작위로 비활성화하여 모델의 강건함에 기여

# dropout은 랜덤하게 뉴런을 비활성화하여 계산량을 줄이는 것으로 자원을 효율적으로 사용하는 반면
# lora에서는 랜덤하게 뉴런을 비활성화 하면서 이후의 레이어에 미치는 영향을 줄이는 방법으로 보인다.

### `bitsandbytes` 파라미터 설정

- `bitsandbytes`는 QLoRA기법을 적용하기 위해 사용되는 8비트 양자회 라이브러리
- 양자화 관련 설정값을 지정할 수 있다.

In [4]:
# 4-bit precision 기반의 모델 로드
use_4bit = True 

# 4비트 기반 모델에 대한 dtype 계산
bnb_4bit_compute_dtype = "bfloat16"

# 양자화 유형(fp4 또는 nf4)
bnb_4bit_quant_type = "nf4"

# 4비트 기 모델에 대해 중첩 양자화 활성화(이중 양자화)
use_nested_quant = False

### `TrainingArguments`파라미터 설정

- 허깅페이스에서 제공하는 라이브러리 모델
- 학습부터 평가까지 한번에 해결할 수 있는 API 제공
- Optimizer의 종류, Learning Rate, Epoch 수, Scheduler와 Half Precicison의 사용 여부 등을 지정 가능

In [5]:
#모델이 예측한 결과와 체크포인트가 저장될 출력 디렉터리
output_dir = "./results" 

# 훈련 에포크 수
num_train_epochs = 1 

# fp16/bf16 학습 활성화(A100으로 bf16을 True로 설정)
fp16 = False   
bf16 = True

# 각 GPU별 훈련용 배치 크기
per_device_train_batch_size = 1

# 각 GPU별 평가용 배치 크기
per_device_eval_batch_size = 1

# 기울기 갱신 전 업데이트 축적 횟수
gradient_accumulation_steps = 8
# 2이상으로 설정 시 기존의 업데이트 내역은 사라지는지
# (기울기 업데이트가 항상 좋은 쪽으로만 업데이트 되진 않는 것을 알고 있기에)

# 그래디언트 체크포인트 활성화
gradient_checkpointing = True  
# 필요할 때만 특정 계층의 기울기를 저장하고 나머지는 버려 메모리의 부담을 줄인다.

# 그래디언트 클리핑을 위한 최대 그래디언트 노름을 설정. 
# 그래디언트 클리핑은 그래디언트의 크기를 제한하여 훈련 중 안정성을 높임.
# Maximum gradient normal (그래디언트 클리핑) 0.3으로 설정
max_grad_norm = 0.3
# 모델이 데이터로부터 학습하는 속도를 조절
# 기울기가 과도하게 커져 gradient exploding 문제를 방지
# 기울기 소실과 반대되는 것으로 역전파를 수행할 때 기울기의 크기가 점점 커져 발산하는 현상

# 초기 학습률 AdamW 옵티마이저
learning_rate = 2e-6
# 학습률을 낮게 잡아 학습하는 속도를 적절히 늦췄다.

# bias/LayerNorm 가중치를 제외하고 모든 레이어에 적용할 Weight decay 값
weight_decay = 0.001
# 모델의 가중치가 너무 큰 값을 가지지 않도록 함
# 이로 오버피팅 현상을 해소 가능하다.

# 옵티마이저 설정
optim = "paged_adamw_32bit"  

# 학습률 스케줄러의 유형 설정, 여기서는 코사인 스케줄러 사용
lr_scheduler_type = "cosine"
# 안정적으로 끊임없이 Loss가 감소하게 한다고 함
# 어떻게?

# 훈련 스텝 수(num_train_epochs 재정의)
max_steps = -1 # epoch 기준으로 한 번만 전체 데이터를 돈다는 뜻

# (0부터 learning rate까지) 학습 초기에 학습률을 점진적으로 증가시키 linear warmup 스텝의 Ratio
warmup_ratio = 0.03  

# 시퀀스를 동일한 길이의 배치로 그룹화, 메모리 절약 및 훈련 속도를 높임
group_by_length = True   

# X 업데이트 단계마다 체크포인트 저장
save_steps = 0

# 매 X 업데이트 스텝 로그
logging_steps = 25  

### SFT 파라미터 값 설정

- 프롬프트 데이터셋을 이용하여 베이스 모델을 지도학습 바탕으로 파인 튜닝하는 기법으로, 일정 토큰에 대한 다음 토큰을 예측하는 형식으로 진행된다.

In [6]:
# 최대 입력 시퀀스 길이 설정
max_seq_length = 512
# 1024로 진행 시 GPU 메모리 부족(GeForce 4060 NoteBook)

# 하나의 입력 시퀀스에 여러 개의 짧은 예시 문장을 한번에 넣어 GPU효율성을 높일 수 있음
packing = False

# GPU를 몇 번 로드할 지 지정
# GPU가 하나기에 0으로 설정 (= 전체 모델 로드)
device_map = {"": 0}

### 데이터 세트 로딩과 데이터 타입 결정

In [7]:
dataset = load_dataset(dataset_name, split="train")

In [8]:
type(dataset)

datasets.arrow_dataset.Dataset

### csv or excel파일을 데이터셋으로 변환

In [9]:
# from datasets import load_dataset

# # CSV 파일 → Dataset
# dataset_csv = load_dataset("csv", data_files="mydata.csv")

# # Excel 파일 → Dataset
# dataset_xlsx = load_dataset("excel", data_files="mydata.xlsx")

# print(dataset_csv["train"])
# print(dataset_xlsx["train"])

In [10]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# 모델 계산에 사용될 데이터 타입 결정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit, # 모델을 4비트로 로드할지 여부
    bnb_4bit_quant_type=bnb_4bit_quant_type, # 양자화 유형 설정
    bnb_4bit_compute_dtype=compute_dtype, # 계산에 사용될 데이터 타입 설정
    bnb_4bit_use_double_quant=use_nested_quant # 중첩 양자화를 사용할지 여부
)

### GPU 호환성 확인

In [11]:

# 만약 GPU가 최소한 버전 8 이상이라면 (major >= 8) bfloat16을 지원한다고 메시지를 출력. 
# bfloat16은 훈련 속도를 높일 수 있는 데이터 타입. 

if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

### 확인용 코드

In [12]:
import torch
print(torch.__version__)         # 2.1.2+cu118
print(torch.version.cuda)        # 11.8
print(torch.backends.cudnn.version())  # cudnn 버전
print(torch.cuda.is_available()) # True 여야 정상
print(torch.cuda.get_device_name(0))  # GPU 모델명 출력

2.2.2+cu121
12.1
8801
True
NVIDIA GeForce RTX 4060 Laptop GPU


In [13]:
print("compute_dtype:", compute_dtype)
print("use_4bit:", use_4bit)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("capability:", torch.cuda.get_device_capability())


compute_dtype: torch.bfloat16
use_4bit: True
cuda available: True
capability: (8, 9)


In [19]:
type(dataset)

datasets.arrow_dataset.Dataset

In [14]:
print(dataset[0])

{'text': '<s>[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país. </s>'}


In [15]:
print(dataset[1])

{'text': '<s>[INST] Самый великий человек из всех живших на планете? [/INST] Для начала нужно выбрать критерии величия человека. Обычно великим называют человека, который внес большой вклад в общество или сильно выделялся на фоне других в своем деле.\n\nНапример, Иосифа Бродского считают великим поэтом, а Иммануила Канта — великим философом. Александр Македонский, известный тем, что собрал в свои владения огромную империю (включавшую Македонию, Грецию, Персию, Египет), в историографии носит имя Александр Великий. Для христиан, скорее всего, самым великим человеком жившим на земле был Иисус Христос, так как он совершил множество благих деяний и совершил подвиг ради человечества. \n\nПри этом, когда мы выдвигаем одну личность на роль великого человека, сразу же находится множество людей, не согласных с этим. Того же Иосифа Бродского, хоть он и получил престижную Нобелевскую премию, некоторые люди считают графоманом и посредственным поэтом. \n\nВ целом, кого считать великим — это самостоя

In [16]:
type(dataset[1])

dict

### 베이스 모델 로딩

In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1     # 분산학습 시 사용하는 옵션, 1이면 기본(나누지 않음)

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 동일한 batch 내에서 입력의 크기를 동일하기 위해서 사용하는 Padding Token을 End of Sequence라고 하는 Special Token으로 사용한다.
tokenizer.pad_token = tokenizer.eos_token   # padding token을 EOS로 설정 Llama2에는 padding token이 없어서 설정
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training. Padding을 오른쪽 위치에 추가한다.
# 문장 + Padding(EOS), fp16학습 시 왼쪽에 패딩이 있으면 버그 유발 가능성 있음

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",    # bias 파라미터는 수정하지 않는다.
    task_type="CAUSAL_LM", # 파인튜닝할 태스크를 Optional로 지정할 수 있는데, 여기서는 CASUAL_LM을 지정하였다.
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,      # 학습 결과 저장 위치
    num_train_epochs=num_train_epochs,  # 학습 반복 횟수
    per_device_train_batch_size=per_device_train_batch_size,    # GPU 하나당 batch 크기
    gradient_accumulation_steps=gradient_accumulation_steps,    # 여러 step에서 grad 쌓아서 효과적으로 큰 batch처럼 학습
    optim=optim,    # optimizer 종류
    save_steps=save_steps,  # 몇 step마다 체크포인트 저장할지
    logging_steps=logging_steps,    # 몇 step마다 로그 출력할지
    learning_rate=learning_rate,    # 학습률
    weight_decay=weight_decay,  # 규제
    fp16=fp16,  # 사용여부
    bf16=bf16,  # 이하동문
    max_grad_norm=max_grad_norm,    # gradient clipping 최대값
    max_steps=max_steps,    # 총 학습 step(epoch대신 step으로 제어 가능)
    warmup_ratio=warmup_ratio,  # 학습 초반에 learning rate를 천천히 올리는 비율
    group_by_length=group_by_length,    # 비슷한 길이의 샘플끼리 묶어서 효율적으로 학습
    lr_scheduler_type=lr_scheduler_type,    # learning rate 스케줄러 종류
    report_to="tensorboard" # 로그 기록을 TensorBoard로 보냄
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,    # LoRA 설정
    dataset_text_field="text",  # 데이터셋에서 텍스트가 들어있는 열 이름 (최근에는 빼는 추세)
    max_seq_length=max_seq_length,  # 토큰 시퀀스 최대 길이(넘어가면 자름)
    tokenizer=tokenizer,    # 토크나이저
    args=training_arguments,    # 학습 설정 전달
    packing=packing,    # 여러 샘플을 이어 붙여 max seq_length를 채워 학습할지 여부(사용 시 효율 상승)
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:11<00:00,  5.99s/it]
c:\Users\hyunwook\anaconda3\envs\llm_practice\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\hyunwook\anaconda3\envs\llm_practice\Lib\site-packages\trl\trainer\sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
c:\Users\hyunwook\anaconda3\envs\llm_practice\Lib\site-packages\trl\trainer\sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
c:\Users\hyunwook\anaconda3\envs\llm_practice\Lib\site-packages\accelerate\accel

### 모델 훈련과 저장

In [ ]:
trainer.train()

# 훈련이 완료된 모델을 'new_model'에 저장 
trainer.model.save_pretrained(new_model) 

In [ ]:
# base_model과 new_model에 저장된 LoRA 가중치를 통합하여 새로운 모델을 생성
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base_model, new_model) # LoRA 가중치를 가져와 기본 모델에 통합
# model = PeftModel.from_pretrained(base_model, "tuned-llama-2-7b-miniguanaco") # 나중에 저장된 파인튜닝된 레이어를 불러오고싶을때 폴더 이름을 적어줘야한다. .safetensors, .json파일 모두 들어있어야함

In [ ]:
model = model.merge_and_unload()

# 사전 훈련된 토크나이저를 다시 로드
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)  

# 토크나이저의 패딩 토큰을 종료 토큰(end-of-sentence token)과 동일하게 설정
tokenizer.pad_token = tokenizer.eos_token  

# 패딩을 시퀀스의 오른쪽에 적용
tokenizer.padding_side = "right"

### 출처

- LLaMA 공식 허깅페이스: meta-llama/Llama-2-7b · Hugging Face
- 튜닝에 사용한 데이터 : https://huggingface.co/datasets/mlabonne/guanaco-llama2-1k
- 참고한 사이트: https://blog.sionic.ai/finetuning_llama